In [ ]:
import sys,os
sys.path.append('..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.strategy import StrategyBase, OptParam
from backtester.swarms import SwarmManager, SwarmRanker

import pandas as pd
import numpy as np
import scipy

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

# Volatility exploration of EXOs

In [ ]:
import os
exos_data = {}
for fn in os.listdir('../mat/'):
    if '.mat' not in fn:
        continue
        
    d, info = matlab.loaddata('../mat/'+fn)
    n = info['underlying']+"_"+info['name']
    print(n)
    print(fn)
    print('----')
    exos_data[n] = d.exo
    

On this figure we have a several different EXOs, with extremely different volatilities.

In [ ]:
exo_df = pd.DataFrame(exos_data)
exo_df.plot()

#### Descriptive stats for every EXO returns

In [ ]:
exo_df.diff().describe()

#### Visual comparison of volatility of returns

In [ ]:
exo_df.diff().plot.box(sym='', rot=34);

# The question: how could we equalize volatility of each EXO

### Linking "R concept" with "Volatility adjusted size"
* I chose median as volatility metric, because it less sensitive to short term volatility bursts than moving average.
* We have only close prices of EXO, that why we are using |exo - exo[-1] | (absolute returns of EXO)

In [ ]:
#
# 60 business days rolling window ~~ 1 quarter of time
#
volatility = exo_df.diff().abs().rolling(100).median()

In [ ]:
figsize(15,10)
volatility.plot();
title("Median volatility of EXOs")

### Our goal is in implementing risk parity or equal risk across unequally volatile products

Let we want to risk 1000 per single volatiliry unit at trade

Then, our positon should be equal to: [AmmountInDollars] / [VolatilityUnit]

Common sense of R is how many shares of each EXO index we should buy or sell to equalize dollar volatility of each position

In [ ]:
R = 1000.0 / volatility

## R-adjusted returns of EXOs analysis

In [ ]:
r_adj_exo_df = exo_df.diff() * R

In [ ]:
r_adj_exo_df.cumsum().plot()

In [ ]:
r_adj_exo_df.describe()

In [ ]:
exo_df.diff().plot.box(sym='', rot=34, title='EXO returns (USD) without adjustments');
r_adj_exo_df.plot.box(sym='', rot=34, title='EXO returns (USD) with volatility adjustments');


## Pitfalls of volatility adjustment method

#### Managing low volatility periods

In my experience, volatility adjustment method has one serious issue. When the volatility of an asset is abnormally low the position size is tending to be higher than usual. In that case we have a probability of substantial loss if volatility returns to normal or high values.

We can manage this problem if we include lower bound constraints in volatility formula calculations.

In [ ]:
v = exo_df.diff().abs()
#
# let our lower bound constraints is 100-period 30% percintile of vola
#
vol_contraints = v.rolling(200).quantile(0.4)

volatility_with_constraints = volatility.apply( lambda x: np.maximum(x, vol_contraints[x.name] ) )

In [ ]:
volatility.EP_BearishCollarBrokenWing.plot()
volatility_with_constraints.EP_BearishCollarBrokenWing.plot()

## Expressing results as R


In fact in the formula R = 1000.0 / volatility, R is not a result of its calculation, but 1000.0 USD value. Which means that we willing to risk about 1000.0 USD per volatility unit.

Thats why we could express trading systems backtesting results as Rs factors

In [ ]:
strategyname = 'strategy_270225'
d, info = matlab.loaddata('../mat/'+strategyname+'.mat')

In [ ]:
d.plot()


In [ ]:
info

In [ ]:
# Position size
# Volatility adjusted
volatility = d.exo.diff().abs().rolling(60).median()
R = 1000.0 
size = R / volatility


slow_ma = d.exo.rolling(5).mean()
fast_ma = d.exo.rolling(6).mean()

short_entry = CrossDown(fast_ma, slow_ma)
short_exit = (CrossUp(fast_ma, slow_ma)) 

direction = -1
pl, inposition = backtester.backtest(d, short_entry, short_exit, direction )
equity, stats = backtester.stats(pl, inposition, positionsize=size)

#X axis, initial equity curve
#Colored blue
x=equity
x.plot()

## Common USD stats

In [ ]:
pd.Series(stats)

## R stats

#### Our net profit is 134.1R and our MaxDrawDown is -44R, Average trade is 1.19R

In [ ]:
(pd.Series(stats) / R).round(2)

### R Equity

In [ ]:
(equity / R).round(2).plot()